# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(131)
initial_params = np.random.random([5])

INITIALIZATION_METHOD = 'Amplitude'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [4]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label in [0, 2, 4, 6, 8]:
        x.append(example)
        y.append(-1)
    else:
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=3)

In [9]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(3), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(3), rotation='Y')

    # First layer
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.CNOT(wires=[0, 1])

    # Second layer
    qml.RY(params[2], wires=1)
    qml.RY(params[3], wires=2)
    qml.CNOT(wires=[1, 2])

    # Third layer
    qml.RY(params[4], wires=2)

    # Measurement
    return qml.expval(qml.PauliZ(2))

## Circuit example

In [10]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.65015361 0.94810917 0.38802889 0.64129616 0.69051205]

Example features: [-1.86150552  1.86874792 -0.01658222 -0.5528763  -0.09764375 -2.42729936
 -0.53099439 -1.02923804]

Expectation value: 0.38083583966240653

 0: ──╭QubitStateVector(M0)──RY(0.65)───╭C────────────────────────────┤     
 1: ──├QubitStateVector(M0)──RY(0.948)──╰X──RY(0.388)──╭C─────────────┤     
 2: ──╰QubitStateVector(M0)──RY(0.641)─────────────────╰X──RY(0.691)──┤ ⟨Z⟩ 
M0 =
[-0.48875467+0.j  0.49065623+0.j -0.00435381+0.j -0.14516254+0.j
 -0.02563723+0.j -0.63730883+0.j -0.13941725+0.j -0.27023551+0.j]



# Accuracy test definition

In [11]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [12]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.4451380552220888
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.4888355342136854
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.5402160864345738
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.5863145258103242
best accuracy so far!
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.6096038415366146
best accuracy so far!
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.612484993997599
best accuracy so far!
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.6105642256902761
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.5983193277310924
epoch 71

epoch 343
epoch 344
epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.6386554621848739
test_accuracies: [0.6105642256902761, 0.5983193277310924, 0.6184873949579832, 0.616326530612245, 0.6165666266506602, 0.6194477791116446, 0.6206482593037215, 0.6235294117647059, 0.617046818727491, 0.6199279711884753, 0.6273709483793517, 0.626890756302521, 0.62953181272509, 0.6304921968787516, 0.6333733493397359, 0.6336134453781512, 0.6261704681872748, 0.6417767106842738, 0.6396158463385354, 0.6386554621848739, 0.6331332533013205, 0.6403361344537815, 0.636014405762305, 0.6376950780312125, 0.6432172869147659, 0.6427370948379352, 0.6388955582232894, 0.6336134453781512, 0.6199279711884753, 0.6386554621848739]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.636014405762305
test_accuracies: [0.5983193277310924, 0.6184873949579832, 0.616326530612245, 0.6165666266506602, 0.6194477791116446, 0.6206482593037215, 0.623529411764

epoch 459
epoch 460
accuracy: 0.6400960384153662
test_accuracies: [0.626890756302521, 0.62953181272509, 0.6304921968787516, 0.6333733493397359, 0.6336134453781512, 0.6261704681872748, 0.6417767106842738, 0.6396158463385354, 0.6386554621848739, 0.6331332533013205, 0.6403361344537815, 0.636014405762305, 0.6376950780312125, 0.6432172869147659, 0.6427370948379352, 0.6388955582232894, 0.6336134453781512, 0.6199279711884753, 0.6386554621848739, 0.636014405762305, 0.6379351740696279, 0.6384153661464587, 0.6336134453781512, 0.6350540216086434, 0.6417767106842738, 0.6218487394957983, 0.6321728691476591, 0.628811524609844, 0.6393757503001201, 0.6400960384153662]
epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.6292917166866747
test_accuracies: [0.62953181272509, 0.6304921968787516, 0.6333733493397359, 0.6336134453781512, 0.6261704681872748, 0.6417767106842738, 0.6396158463385354, 0.6386554621848739, 0.6331332533013205, 0.640336134453

epoch 572
epoch 573
epoch 574
epoch 575
epoch 576
epoch 577
epoch 578
epoch 579
epoch 580
accuracy: 0.6364945978391356
test_accuracies: [0.6376950780312125, 0.6432172869147659, 0.6427370948379352, 0.6388955582232894, 0.6336134453781512, 0.6199279711884753, 0.6386554621848739, 0.636014405762305, 0.6379351740696279, 0.6384153661464587, 0.6336134453781512, 0.6350540216086434, 0.6417767106842738, 0.6218487394957983, 0.6321728691476591, 0.628811524609844, 0.6393757503001201, 0.6400960384153662, 0.6292917166866747, 0.6290516206482593, 0.6350540216086434, 0.637454981992797, 0.6331332533013205, 0.6362545018007203, 0.6348139255702281, 0.6384153661464587, 0.6345738295318127, 0.6338535414165667, 0.6312124849939976, 0.6364945978391356]
epoch 581
epoch 582
epoch 583
epoch 584
epoch 585
epoch 586
epoch 587
epoch 588
epoch 589
epoch 590
accuracy: 0.62953181272509
test_accuracies: [0.6432172869147659, 0.6427370948379352, 0.6388955582232894, 0.6336134453781512, 0.6199279711884753, 0.6386554621848739, 0

In [13]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [ 1.85644118  1.93530235  2.87485682  1.02825625 -0.00644162]


# Testing

In [14]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.6452380952380952


In [15]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 8.684840202331543
training time: 673.0552532672882
test time: 27.717413425445557
total time: 709.4575068950653
